In [1]:
import numpy as np
import scipy
import a_mps

### Part a
Read the code in the file a_mps.py. This file defines the class MPS in an object-oriented
approach. In short, defining the class is defining a “type” which collects data in
attributes (e.g. MPS.Bs, MPS.L) and has methods (e.g. MPS.site_expectation_value)
which can use the attributes (referenced with the special first argument self) for
calculations. Generate an instance of the MPS class representing the state |↑↑ . . . ↑⟩
with the function init_spinup_MPS, for the start with L = 14 sites. Check that the
(site) expectation values of the operators σz
and σx  give the expected values.

In [7]:
state = a_mps.init_spinup_MPS(14)
sigmax = np.array([[0,1],[1,0]])
sigmaz = np.array([[1,0],[0,-1]])

sigmaxmean = state.site_expectation_value(sigmax)
sigmazmean = state.site_expectation_value(sigmaz)



## Part b
Write a function similar to init_spinup_MPS, but initialize an MPS for the state
|→→ · · · →⟩. Check the expectation values again.
Hint: This state is also a product state of |→⟩ = 1√2 (|↑⟩+|↓⟩), so the singular values
remain the same and the shape of each B is still (1,2,1)

In [8]:
import importlib
importlib.reload(a_mps)

stateright = a_mps.init_spinright_MPS(14)

### Part c)
Read the file b_model.py. It defines a class representing the transverse field ising
model for a given choice of coupling parameters. Calculate the energy for L =
14, J = 1 and g ∈ { 0.5, 1, 1.5 } for each of the above defined two product states.

In [9]:
import b_model
L = 14
g = [0.5,1,1.5]
J = 1.
H_model = {}
Energyhor = {}
Energyup = {}

for gs in g:
    H_model[gs] = b_model.TFIModel(L,J,gs)
    Energyhor[gs] = H_model[gs].energy(stateright)
    Energyup[gs] = H_model[gs].energy(state)



In [10]:
Energyup[0.5]

-7.0

d) Read the file c_tebd.py, which implements the time evolving block decimation. Call
the function example_TEBD_gs_finite, which performs an imaginary time evolution
to project onto the ground state. (As we will see next week, DMRG is an better
alternative to find ground states, but since we only discussed TEBD in class so far,
we will use this method.)

In [13]:
import c_tebd

E, psi, model = c_tebd.example_TEBD_gs_finite(L,J,g=1.5)

finite TEBD, (imaginary time evolution)
L=14, J=1.0, g=1.50
dt = 0.10000: E = -23.1210722544100
dt = 0.01000: E = -23.2133644786914
dt = 0.00100: E = -23.2220037685472
dt = 0.00010: E = -23.2228606166766
dt = 0.00001: E = -23.2229463553360
final bond dimensions:  [2, 4, 8, 13, 17, 18, 18, 18, 17, 13, 8, 4, 2]
Exact diagonalization: E = -23.2229594341174
relative error:  5.631832304556125e-07


e) Global quench. Calculate the real time evolution of the spin-up state, |ψ(t)⟩ =
e−iHt |↑ . . . ↑⟩ for L = 14, J = 1, g = 1.5. As a first choice, use the parameters
chi_max = 30, eps=1.e-10. Evolve up to time t = 10J. Measure and plot the total
magnetization P σz
i and the half-chain entropy as a function of time t.
Hint: Don’t forget the imaginary i for the time step when calculating U_bonds.
For the measurements, you can use the methods MPS.site_expectation_value and
MPS.entropy.

In [16]:
from scipy.linalg import expm

def calc_U_bonds_real(model, dt):
    """Given a model, calculate ``U_bonds[i] = expm(-dt*model.H_bonds[i])``.

    Each local operator has legs (i out, (i+1) out, i in, (i+1) in), in short ``i j i* j*``.
    Note that no imaginary 'i' is included, thus real `dt` means imaginary time evolution!
    """
    H_bonds = model.H_bonds
    d = H_bonds[0].shape[0]
    U_bonds = []
    for H in H_bonds:
        H = np.reshape(H, [d * d, d * d]) #reshape it as a matrix to calculate the time evolution
        U = expm(-1j*dt * H)
        U_bonds.append(np.reshape(U, [d, d, d, d])) # reshape it as a tensor with 4 legs so that I can "multiply" it with MPS
        #U_bonds is list of all infinitesimal time evolutions for each pair of sites j,j+1
    return U_bonds

L = 14
J = 1
g = 1.5
dt = 0.01
t = 10*J
N_steps = int(t//dt)

model = b_model.TFIModel(L, J=J, g=g)
psi = a_mps.init_spinup_MPS(L)
U_bonds = calc_U_bonds_real(model, dt)
c_tebd.run_TEBD(psi, U_bonds, N_steps, chi_max=30, eps=1.e-10)

M = 0
for i in range(L):
    M += psi.site_expectation_value(sigmaz)[i]

S = psi.entanglement_entropy


TypeError: 'method' object is not subscriptable